In [ ]:
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [ ]:
#%cd /gdrive/MyDrive/Dataset

In [ ]:
#!cp -r '/gdrive/.shortcut-targets-by-id/1ijYOvNHozPmAD9nWgnhfWlNyatICd8Pv/Dataset/.' '/gdrive/Shared drives/BTP_Group2/DATASET2'

In [ ]:
############Importing Necessary Modules#########
import string
import numpy as np
from PIL import Image
import os
from pickle import dump, load
import numpy as np
from keras.applications.xception import Xception, preprocess_input
from keras.preprocessing.image import load_img, img_to_array
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.layers.merge import add
from keras.models import Model, load_model
from keras.layers import Input, Dense, LSTM, Embedding, Dropout
# small library for seeing the progress of loops.
from tqdm import tqdm_notebook as tqdm
tqdm().pandas()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:18: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


/usr/local/lib/python3.7/dist-packages/tqdm/std.py:658: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


# **Paired Data Preprocessing To get Image-Caption Feature Vectors**

In [ ]:
##### Loading a Text File Into Memory.
def load_doc(filename):
    # Opening the file as read only
    file = open(filename, 'r')
    text = file.read()
    file.close()
    return text



##### Store all 5 captions of each image
def all_img_captions(filename):
    file = load_doc(filename)
    captions = file.split('\n')
    descriptions ={}
    for caption in captions[:-1]:
        img, caption = caption.split('\t')
        if img[:-2] not in descriptions:
            descriptions[img[:-2]] = [ caption ]
        else:
            descriptions[img[:-2]].append(caption)

    return descriptions


##### Preprocessing data,We will remove punctuation,",","-" and other non alphanumeric characters.
def cleaning_text(captions):
    table = str.maketrans('','',string.punctuation)
    for img,caps in captions.items():
        for i,img_caption in enumerate(caps):
            img_caption.replace("-"," ")
            desc = img_caption.split()
            #converts to lowercase
            desc = [word.lower() for word in desc]
            #remove punctuation from each token
            desc = [word.translate(table) for word in desc]
            #remove hanging 's and a 
            desc = [word for word in desc if(len(word)>1)]
            #remove tokens with numbers in them
            desc = [word for word in desc if(word.isalpha())]
            #convert back to string
            img_caption = ' '.join(desc)
            captions[img][i]= img_caption
    return captions


##### Building vocabularry of all unique words
def text_vocabulary(descriptions):
    # build vocabulary of all unique words
    vocab = set()
    for key in descriptions.keys():
        [vocab.update(d.split()) for d in descriptions[key]]
    return vocab


##### All descriptions in one file 
def save_descriptions(descriptions, filename):
    lines = list()
    for key, desc_list in descriptions.items():
        for desc in desc_list:
            lines.append(key + '\t' + desc )
    data = "\n".join(lines)
    file = open(filename,"w")
    file.write(data)
    file.close()


#####Preparing all the paths of datasets required.
#Contains all text files containing image name and captions related with them.
dataset_text = "/gdrive/Shared drives/BTP_Group2/DATASET2/Flickr_8k_text"

#Contains 8000 images.
dataset_images = "/gdrive/Shared drives/BTP_Group2/DATASET2/Flicker8k_Dataset"
num1=0
for item in os.listdir(dataset_images):
  num1=num1+1
print('Total No of Images :',num1)
##### Preparing our text data
filename = dataset_text + "/" + "Flickr8k.token.txt"

descriptions = all_img_captions(filename)

print("Length of descriptions =" ,len(descriptions))
num=0
for key in descriptions.keys():
  for item in descriptions[key]:
    num=num+1

print("Number of captions =",num)

cnt=0

for key in descriptions.keys():
  if(cnt>5):
    break
  print(key,end=" ")
  print(descriptions[key])
  cnt=cnt+1


#cleaning the descriptions
clean_descriptions = cleaning_text(descriptions)

cnt=0
print("\n\nClean descriptions\n")
for key in clean_descriptions.keys():
  if(cnt>5):
    break
  print(key,end=" ")
  print(clean_descriptions[key])
  cnt=cnt+1


#building vocabulary 
vocabulary = text_vocabulary(clean_descriptions)

#Vocabulary Stores all unique words occuring in all captions

print("\nLength of vocabulary = ", len(vocabulary))

#saving each description to file 
save_descriptions(clean_descriptions, "descriptions.txt")



#2048 feature vector
# features = extract_features(dataset_images)
#dump(features, open("/content/drive/My Drive/Dataset/features.p","wb"))


#load the data 
def load_photos(filename):
    file = load_doc(filename)
    photos = file.split("\n")[:-1]
    return photos


def load_clean_descriptions(filename, photos): 
    #loading clean_descriptions
    file = load_doc(filename)
    descriptions = {}
    for line in file.split("\n"):
        words = line.split()
        if len(words)<1 :
            continue
        image, image_caption = words[0], words[1:]
        if image in photos:
            if image not in descriptions:
                descriptions[image] = []
            desc = '<start> ' + " ".join(image_caption) + '<end>'
            descriptions[image].append(desc)
    return descriptions



# def load_features(photos):
#     #loading all features
#     all_features = load(open("/content/drive/My Drive/Dataset/features.p","rb"))
#     #selecting only needed features
#     features = {k:all_features[k] for k in photos}
#     return features

####TRAINING DATASET

filename = dataset_text + "/" + "Flickr_8k.trainImages.txt"

#Training dataset named Flickr_8k.trainImages.txt contains name of 6000 images which are being used for training our model. 
train_imgs = load_photos(filename)

train_descriptions = load_clean_descriptions("descriptions.txt", train_imgs)

# train_features = load_features(train_imgs)


#converting dictionary to clean list of descriptions

def dict_to_list(descriptions):
    all_desc = []
    for key in descriptions.keys():
        [all_desc.append(d) for d in descriptions[key]]
    return all_desc


#creating tokenizer class 
#this will vectorise text corpus
#each integer will represent token in dictionary

from keras.preprocessing.text import Tokenizer

def create_tokenizer(descriptions):
    desc_list = dict_to_list(descriptions)
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(desc_list)
    return tokenizer


# give each word an index, and store that into tokenizer.p pickle file

tokenizer = create_tokenizer(train_descriptions)

dump(tokenizer, open('tokenizer.p', 'wb'))

vocab_size = len(tokenizer.word_index) + 1

#calculate maximum length of descriptions

def max_length(descriptions):
    desc_list = dict_to_list(descriptions)
    return max(len(d.split()) for d in desc_list)

max_length = max_length(descriptions)
num=0
for key, des_list in descriptions.items():
  for desc in des_list:
    seq = tokenizer.texts_to_sequences([desc])[0]
    print(seq)
  if num==0:
    break

print("\nMax length of a caption in given dataset= ",max_length)




Total No of Images : 8093
Length of descriptions = 8092
Number of captions = 40460
1000268201_693b08cb0e.jpg ['A child in a pink dress is climbing up a set of stairs in an entry way .', 'A girl going into a wooden building .', 'A little girl climbing into a wooden playhouse .', 'A little girl climbing the stairs to her playhouse .', 'A little girl in a pink dress going into a wooden cabin .']
1001773457_577c3a7d70.jpg ['A black dog and a spotted dog are fighting', 'A black dog and a tri-colored dog playing with each other on the road .', 'A black dog and a white dog with brown spots are staring at each other in the street .', 'Two dogs of different breeds looking at each other on the road .', 'Two dogs on pavement moving toward each other .']
1002674143_1b742ab4b8.jpg ['A little girl covered in paint sits in front of a painted rainbow with her hands in a bowl .', 'A little girl is sitting in front of a large painted rainbow .', 'A small girl in the grass plays with fingerpaints in fron

In [ ]:
# #####  FOR ZX AND ZY

# from keras.models import Sequential
# from keras.layers import Dense, Concatenate
# import keras.backend as K

# #For Understanding.
# from keras.preprocessing import image
# model1 = Xception( include_top=False, pooling='avg' )

# ##### Function to return image feature vector of size 256
# def preprocess_image_and_return_Zx(filename):
#   # %pylab inline
#   # import matplotlib.pyplot as plt
#   # import matplotlib.image as mpimg
#   # img = mpimg.imread(filename)
#   # imgplot = plt.imshow(img)
#   # plt.show()
#   imag = Image.open(filename)
#   imag = imag.resize((224,224))
#   x = image.img_to_array(imag) 
#   x = np.expand_dims(x, axis=0) 
#   x= x/127.5
#   x = x - 1.0 
#   y=model1.predict(x)
#   fe1 = Dropout(0.5)(y)#TO prevent overfitting of neural network.
#   fe2 = Dense(256, activation='relu')(fe1)
#   return fe2


# ##### Function to return caption feature vector of size 256
# def return_Zy_for_caption(desc):
#   sequence = tokenizer.texts_to_sequences([desc])[0]
#   sequence = pad_sequences([sequence], maxlen=max_length)
#   # print(desc)
#   ####Token extracted   #####
#   se1 = Embedding(vocab_size, 256, mask_zero=True)(sequence)
#   se2 = Dropout(0.5)(se1)
#   se3 = LSTM(256)(se2)
#   return se3



# #Description is a dict having key as image name and value as list of captions.


# train_list_X=[]      # List of concatenated feature vectors
# train_list_Y=[]      # List of labels of pair, if paired label=1 else label=0;
# #### SET NUMBER OF SAMPLES YOU WANT TO TAKE FOR PAIRED SAMPLES AS "num" #####
# num=3000 #3000 samples of paired data
# n1=num
# for key, des_list in train_descriptions.items():
#   directory= "/gdrive/Shared drives/BTP_Group2/DATASET2/Flicker8k_Dataset"
#   #####Extracting feature vector of image#######
#   filename = directory + "/" + key
  
#   ####  Z[x]  ######## 
#   Zx=preprocess_image_and_return_Zx(filename)
#   num-=1

#   for desc in des_list:
#     #### To Generate Tokens ####
#     ########## Z[y] .#############
#     Zy=return_Zy_for_caption(desc)
#     temp= Concatenate()([Zx, Zy])
#     train_list_X.append(temp)
#     print(len(train_list_X))
#     train_list_Y.append([1])
#   if num==0:
#     break


# ###### Adding unpaired samples to the train list ######
# num=3000 #3000 samples
# list_image_key=[]
# list_caption_image=[]
# n=0
# for key, des_list in train_descriptions.items():
#   n+=1
#   if n<n1:
#     continue
#   list_image_key.append(key)
#   for desc in des_list:
#     list_caption_image.append(desc)
#   num-=1
#   if num==0:
#     break


# #Reverse Image key list.

# new_list_key=[]
# n=3000
# for i in range(3000):
#   new_list_key.append(list_image_key[n-1-i])

# for i in range(3000):
#   key=new_list_key[i]
#   directory= "/gdrive/Shared drives/BTP_Group2/DATASET2/Flicker8k_Dataset"
#   #####Extracting feature vector of image#######
#   filename = directory + "/" + key
#   Zx=preprocess_image_and_return_Zx(filename)
#   for j in range(5*i,5*(i+1)):
#     Zy=return_Zy_for_caption(list_caption_image[j])
#     temp= Concatenate()([Zx, Zy])
#     train_list_X.append(temp)
#     print(len(train_list_X))
#     train_list_Y.append([0])


# print(len(train_list_X))
# print(len(train_list_Y))

In [ ]:
import csv

In [ ]:
# file = open('/gdrive/Shared drives/BTP_Group2/DATASET2/g4g.csv', 'w+', newline ='')
# with file:    
#     write = csv.writer(file)
#     write.writerows(train_list_X)

# **Training Discriminator**

In [ ]:
# import numpy as np
# train_list_Y = np.array(train_list_Y)

# train_list_X=np.array(train_list_X)

In [ ]:
# with open('/gdrive/Shared drives/BTP_Group2/test.npy', 'wb') as f:
#     np.save(f, train_list_X)
#     np.save(f, train_list_Y)


In [ ]:
with open('/gdrive/Shared drives/BTP_Group2/test.npy', 'rb') as f:
    train_list_X = np.load(f)
    train_list_Y = np.load(f)

In [ ]:
np.take(train_list_X,np.random.rand(train_list_X.shape[0]).argsort(),axis=0,out=train_list_X)
np.take(train_list_Y,np.random.rand(train_list_Y.shape[0]).argsort(),axis=0,out=train_list_Y)


array([[1],
       [0],
       [1],
       ...,
       [1],
       [1],
       [1]])

In [ ]:
train_list_X.shape


(30000, 1, 512)

In [ ]:
train_list_Y.shape

(30000, 1)

In [ ]:
####### Training Discriminator ######

from keras.models import Sequential
from keras.layers import *
import tensorflow as tf

ndf = 64

model2 = Sequential()
model2.add(Reshape((1,1,512)))
model2.add(Conv2D( ndf, (4, 4), strides=(2, 2), padding="same", input_shape=(1,1, 512)))
model2.add(LeakyReLU(alpha=0.3))

model2.add(Conv2D( ndf*2, (4, 4), padding="same", strides=(2, 2)))
model2.add(BatchNormalization())
model2.add(LeakyReLU(alpha=0.1))

model2.add(Conv2D( ndf*4, (4, 4), padding="same", strides=(2, 2)))
model2.add(BatchNormalization())
model2.add(LeakyReLU(alpha=0.15))

model2.add(Conv2D( ndf*8, (4, 4), padding="same", strides=(2, 2)))
model2.add(BatchNormalization())
model2.add(LeakyReLU(alpha=0.25))

model2.add(Conv2D( ndf*2, (1, 1), padding="valid", strides=(1, 1)))
model2.add(BatchNormalization())
model2.add(LeakyReLU(alpha=0.2))

model2.add(Conv2D( ndf*2, (3, 3), padding="same", strides=(1, 1)))
model2.add(BatchNormalization())
model2.add(LeakyReLU(alpha=0.2))

model2.add(Conv2D( ndf*8, (1, 1), padding="valid", strides=(1, 1)))
model2.add(BatchNormalization())
model2.add(LeakyReLU(alpha=0.3))

# layers.Conv2D( ndf*8, (1, 1), padding="valid"),
#         layers.BatchNormalization(),
#         layers.LeakyReLU(alpha=0.2),
#         layers.Conv2D(1, (4, 4), strides=(1, 1), padding="valid"),
#         layers.Activation('sigmoid')

model2.add(Conv2D( ndf*8, (1, 1), padding="same"))
model2.add(BatchNormalization())
model2.add(LeakyReLU(alpha=0.2))

model2.add(Conv2D( 1, (1, 1), padding="same", activation="relu"))
model2.add(Flatten())
model2.add(Dense(1, activation='relu'))
model2.add(Activation('sigmoid'))

model2.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

checkpoint_path = "/gdrive/Shared drives/BTP_Group2/DATASET2/weights_discriminator.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# Create a callback that saves the model's weights
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)
# fit the keras model on the dataset
model2.fit(train_list_X, train_list_Y, epochs=25,batch_size=64,validation_split=0.3, shuffle=True,callbacks=[cp_callback])
os.listdir(checkpoint_dir)              

# discriminator_n2 = keras.Sequential(
#     [
#         keras.Input(shape=(4, 4, 576)),
#         
#     ], 
#     name="Discrim_n2",
# ) 

Epoch 1/25
329/329 [==============================] - 10s 27ms/step - loss: 0.6931 - accuracy: 0.5052 - val_loss: 0.6931 - val_accuracy: 0.4959

Epoch 00001: saving model to /gdrive/Shared drives/BTP_Group2/DATASET2/weights_discriminator.ckpt
Epoch 2/25
329/329 [==============================] - 8s 25ms/step - loss: 0.6931 - accuracy: 0.5058 - val_loss: 0.6931 - val_accuracy: 0.4959

Epoch 00002: saving model to /gdrive/Shared drives/BTP_Group2/DATASET2/weights_discriminator.ckpt
Epoch 3/25
329/329 [==============================] - 8s 24ms/step - loss: 0.6931 - accuracy: 0.5009 - val_loss: 0.6931 - val_accuracy: 0.4959

Epoch 00003: saving model to /gdrive/Shared drives/BTP_Group2/DATASET2/weights_discriminator.ckpt
Epoch 4/25
329/329 [==============================] - 8s 24ms/step - loss: 0.6931 - accuracy: 0.4967 - val_loss: 0.6931 - val_accuracy: 0.4959

Epoch 00004: saving model to /gdrive/Shared drives/BTP_Group2/DATASET2/weights_discriminator.ckpt
Epoch 5/25
329/329 [===========

['model_4 (1).h5',
 'features.p',
 'Flicker8k_Dataset',
 'model_4.h5',
 'Flickr_8k_text',
 'g4g.csv',
 'weights.ckpt.index',
 'weights.ckpt.data-00000-of-00001',
 'weights_gen2.ckpt.index',
 'weights_gen2.ckpt.data-00000-of-00001',
 'weights_discriminator (1).ckpt.data-00000-of-00001',
 'weights_discriminator.ckpt.index',
 'weights_discriminator.ckpt.data-00000-of-00001',
 'checkpoint']

# **Caption-to-Image GAN**

In [ ]:
####### Preparing Data For Caption to image Generator############


from keras.models import Sequential
from keras.layers import Dense, Concatenate
import keras.backend as K




##### Function to return caption feature vector of size 256
def return_Zy_for_caption(desc):
  sequence = tokenizer.texts_to_sequences([desc])[0]
  sequence = pad_sequences([sequence], maxlen=max_length)
  # print(desc)
  ####Token extracted   #####
  se1 = Embedding(vocab_size, 256, mask_zero=True)(sequence)
  se2 = Dropout(0.5)(se1)
  se3 = LSTM(256)(se2)
  return se3



#Description is a dict having key as image name and value as list of captions.


GAN_Train_list=[]      ###List of Captions Embeddings

num=6000 #total 6000X5=30000 samples captions
for key, des_list in train_descriptions.items():
  for desc in des_list:
    #### To Generate Tokens ####
    ########## Z[y] .#############
    Zy=return_Zy_for_caption(desc)
    GAN_Train_list.append(Zy)
  print(num)
  num=num-1


print(len(GAN_Train_list))


In [ ]:
import numpy as np
GAN_Train_list2=[]
GAN_Train_list2 = GAN_Train_list
GAN_Train_list2 = np.array(GAN_Train_list2)
with open('/gdrive/Shared drives/BTP_Group2/test2.npy', 'wb') as f:
  np.save(f, GAN_Train_list2)


In [ ]:
with open('/gdrive/Shared drives/BTP_Group2/test2.npy', 'rb') as f:
    GAN_Train_list2 = np.load(f)
GAN_Train_list2.shape  

(30000, 1, 256)

In [ ]:
GAN_Train_list2 = np.expand_dims(GAN_Train_list2, axis=1)
GAN_Train_list2.shape 

(30000, 1, 1, 256)

In [ ]:
#####Adding a noice vector to each caption feature vector######
import numpy as np
GAN_Train_list1 = GAN_Train_list
GAN_Train_list = []
i=0
for item in GAN_Train_list1:
  item = np.array(item)
  noise = np.random.normal(0, .1, (1,256))
  # GAN_Train_list[i] = Concatenate()([item, noise])
  GAN_Train_list.append(tf.concat([item, noise], axis = -1))
  i=i+1
  

In [ ]:
# GAN_Train_list

In [ ]:
GAN_Train_list=np.array(GAN_Train_list)
GAN_Train_list.shape

(30000, 1, 512)

In [ ]:
with open('/gdrive/Shared drives/BTP_Group2/test1.npy', 'wb') as f:
    np.save(f, GAN_Train_list)
    


In [ ]:
with open('/gdrive/Shared drives/BTP_Group2/test1.npy', 'rb') as f:
    GAN_Train_list = np.load(f)
GAN_Train_list.shape    

(30000, 1, 512)

In [ ]:
####### Training Generator ######
ngf = 64

from keras.models import Sequential
from keras import layers
import tensorflow as tf
import keras 

model1 = keras.Sequential(
    [
        keras.Input(shape=(1, 1, 512)),
        layers.Conv2DTranspose( ngf*8, (4, 4), strides=(1, 1), padding="same"),
        layers.BatchNormalization(),
        layers.ReLU(),
     
        layers.Conv2D(ngf*2, (1, 1), padding="same"),
        layers.BatchNormalization(),
        layers.ReLU(),
     
        layers.Conv2D(ngf*2, (3, 3), padding="same", strides=(1, 1)),
        layers.BatchNormalization(),
        layers.ReLU(),
     
        layers.Conv2D(ngf*8, (3, 3), padding="same", strides=(1, 1)),
        layers.BatchNormalization(),
        layers.ReLU(),
     
        layers.Conv2DTranspose( ngf*4, (4, 4), strides=(1, 1), padding="same"),   
        layers.BatchNormalization(),
        layers.ReLU(),
     
        layers.Conv2D(ngf, (1, 1), padding="same"),
        layers.BatchNormalization(),
        layers.ReLU(),
     
        layers.Conv2D(ngf, (3, 3), padding="same", strides=(1, 1)),
        layers.BatchNormalization(),
        layers.ReLU(),
     
        layers.Conv2D(ngf*4, (3, 3), padding="same", strides=(1, 1)),
        layers.BatchNormalization(),
        layers.ReLU(),
     
        layers.Conv2DTranspose( ngf*2, (4, 4), strides=(2, 2), padding="same"),  
        layers.Conv2DTranspose( ngf*2, (4, 4), strides=(2, 2), padding="same"),  
        layers.Conv2DTranspose( ngf*2, (4, 4), strides=(2, 2), padding="same"),                 
        layers.Conv2DTranspose( ngf*2, (4, 4), strides=(2, 2), padding="same"),   
        layers.BatchNormalization(),
        layers.ReLU(),      

        layers.Conv2DTranspose( ngf, (4, 4), strides=(2, 2), padding="same"),   
        layers.BatchNormalization(),
        layers.ReLU(),      
  
        layers.Conv2DTranspose( 3, (1, 1), strides=(2, 2), padding="same"),   
        layers.BatchNormalization(),
        layers.ReLU()

    ],
    name='Generator1',
)


In [ ]:
model1.summary()

In [ ]:
GAN_Train_list = np.expand_dims(GAN_Train_list, axis=1)

In [ ]:
GAN_Train_list.shape

(30000, 1, 1, 512)

In [ ]:
from tensorflow.keras.layers import AveragePooling2D
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Input
#model_vgg = VGG16(weights="imagenet", include_top=False,
#	input_tensor=Input(shape=(64,64,3)))
model_Xception = Xception( include_top=False, pooling='avg' )
def funtion_to_reshape(input):
  y=model_Xception(input)
  fe1 = Dropout(0.5)(y) #TO prevent overfitting of neural network.
  fe2 = Dense(256, activation='relu')(fe1)
  return fe2

83689472/83683744 [==============================] - 0s 0us/step


In [ ]:
g_optimizer = keras.optimizers.Adam(learning_rate=0.0004)

# Instantiate a loss function.
loss_fn = keras.losses.BinaryCrossentropy(from_logits=True)

def train_generator(batch, epoch):
    
    
    batch_size = 64    
    with tf.GradientTape() as tape:
      # Decode them to fake images
      generated_images = model1(batch)
      generated_images=funtion_to_reshape(generated_images)
      #tf.Reshape(generated_images, (batch_size,1, 1, 256))
      #print(generated_images.shape)
      generated_images = tf.expand_dims(generated_images, axis=1)
      generated_images = tf.expand_dims(generated_images, axis=1)
      #print(generated_images.shape)
      if (epoch == 468):
        Z = tf.convert_to_tensor(GAN_Train_list2[epoch*batch_size:])
        labels = tf.ones((48, 1), dtype=float)  
      else:
        Z = tf.convert_to_tensor(GAN_Train_list2[epoch*batch_size:(epoch+1)*(batch_size)])
        labels = tf.ones((batch_size, 1), dtype=float)
      generate = tf.concat([generated_images, Z], axis = -1)
      #input - bs*1*1*512
    
      predictions = model2(generate)
      loss = loss_fn(labels, predictions)
    grads = tape.gradient(loss, model1.trainable_weights)
    g_optimizer.apply_gradients(zip(grads, model1.trainable_weights))
    
    return loss

In [ ]:
batch_size = 64
dataset = tf.data.Dataset.from_tensor_slices(GAN_Train_list)
dataset = dataset.shuffle(buffer_size=1024).batch(batch_size)
# dataset = dataset.map

In [ ]:

def fit():
  
    # Losses & scores
    losses_g = []
    
    epochs = 25
    for epoch in range(epochs):
      for i, batch_X in enumerate(dataset):
        
      # for batch_X in ds.batch_iterator(batch_size=64):
        
        loss_g = train_generator(batch_X, i)
        print(i)
        #i = i + 1
        losses_g.append(loss_g)
        
      print("Loss After epoch")
      print(loss_g)  
      #print("Epoch [{}/{}], loss_g: {:.4f},  real_score: {:.4f}, fake_score: {:.4f}".format(
            #epoch+1, epochs, loss_g)) #loss_d, real_score, fake_score))
        # Save generated images
        #save_samples(epoch+start_idx, fixed_latent, show=False)
    
    return losses_g #, losses_d, real_scores, fake_scores

In [ ]:
loss = fit()

Streaming output truncated to the last 5000 lines.
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277
278
279
280
281
282
283
284
285
286
287
288
289
290
291
292
293
294
295
296
297
298
299
300
301
302
303
304
305
306
307
308
309
310
311
312
313
314
315
316
317
318
319
320
321
322
323
324
325
326
327
328
329
330
331
332
333
334
335
336
337
338
339
340
341
342
343
344
345
346
347
348
349
350
351
352
353
354
355
356
357
358
359
360
361
362
363
364
365
366
367
368
369
370
371
372
373
374
375
376
377
378
379
380
381
382
383
384
385
386
387
388
389
390
391
392
393
394
395
396
397
398
399
400
401
402
403
404
405
406
407
408
409
410
411
412
413
414
415
416
417
4

In [ ]:
model1.save_weights('/gdrive/Shared drives/BTP_Group2/checkpoints/my_checkpoint')


In [ ]:
model1.load_weights('/gdrive/Shared drives/BTP_Group2/checkpoints/my_checkpoint')

In [ ]:
model2.summary()

In [ ]:
# print(model.predict(train_list_X[16789]))

In [ ]:
# print(model.predict(test_list_X[251]))
# print(test_list_Y[251])

In [ ]:
import tensorflow as tf


# **Image-to-Caption GAN**

In [ ]:
#####  FOR ZX 
import tensorflow as tf

from keras.models import Sequential
from keras.layers import Dense, Concatenate
import keras.backend as K

#For Understanding.
from keras.preprocessing import image
model1 = Xception( include_top=False, pooling='avg' )

##### Function to return image feature vector of size 256
def preprocess_image_and_return_Zx(filename):
  # %pylab inline
  # import matplotlib.pyplot as plt
  # import matplotlib.image as mpimg
  # img = mpimg.imread(filename)
  # imgplot = plt.imshow(img)
  # plt.show()
  imag = Image.open(filename)
  imag = imag.resize((224,224))
  x = image.img_to_array(imag) 
  x = np.expand_dims(x, axis=0) 
  x= x/127.5
  x = x - 1.0 
  y=model1.predict(x)
  fe1 = Dropout(0.5)(y)#TO prevent overfitting of neural network.
  fe2 = Dense(256, activation='relu')(fe1)
  return fe2


# ##### Function to return caption feature vector of size 256
# def return_Zy_for_caption(desc):
#   sequence = tokenizer.texts_to_sequences([desc])[0]
#   sequence = pad_sequences([sequence], maxlen=max_length)
#   # print(desc)
#   ####Token extracted   #####
#   se1 = Embedding(vocab_size, 256, mask_zero=True)(sequence)
#   se2 = Dropout(0.5)(se1)
#   se3 = LSTM(256)(se2)
#   return se3



#Description is a dict having key as image name and value as list of captions.


GAN2_Train_list=[] 
GAN2_Train_list1=[]     # List of concatenated feature vectors
    # List of labels of pair, if paired label=1 else label=0;
#### SET NUMBER OF SAMPLES YOU WANT TO TAKE FOR PAIRED SAMPLES AS "num" #####
num=5952 
n1=num
for key, des_list in train_descriptions.items():
  directory= "/gdrive/Shared drives/BTP_Group2/DATASET2/Flicker8k_Dataset"
  #####Extracting feature vector of image#######
  filename = directory + "/" + key
  
  ####  Z[x]  ######## 
  Zx=preprocess_image_and_return_Zx(filename)
  GAN2_Train_list1.append(Zx)
  num-=1
  noise = np.random.normal(0, .1, (1,256))
  temp= tf.concat([np.array(Zx), noise], axis=-1)
  GAN2_Train_list.append(temp)
  print(len(GAN2_Train_list)) 
  if num==0:
    break






83689472/83683744 [==============================] - 1s 0us/step
1
2
3
4
5


KeyboardInterrupt: ignored

In [ ]:
import numpy as np
GAN2_Train_list = np.array(GAN2_Train_list)
GAN2_Train_list1 = np.array(GAN2_Train_list1)


In [ ]:
with open('/gdrive/Shared drives/BTP_Group2/test3.npy', 'wb') as f:
  np.save(f, GAN2_Train_list)
  np.save(f, GAN2_Train_list1)



In [ ]:
with open('/gdrive/Shared drives/BTP_Group2/test3.npy', 'rb') as f:
  GAN2_Train_list = np.load(f)
  GAN2_Train_list1 = np.load(f)


In [ ]:
GAN2_Train_list.shape


(5952, 1, 512)

In [ ]:
GAN2_Train_list1.shape

(5952, 1, 256)

In [ ]:
GAN2_Train_list = np.expand_dims(GAN2_Train_list, axis=1)
GAN2_Train_list1 = np.expand_dims(GAN2_Train_list1, axis=1)

In [ ]:
GAN2_Train_list.shape
GAN2_Train_list1.shape

(5952, 1, 1, 256)

In [ ]:
from keras.models import Sequential
from keras import layers
import tensorflow as tf
import keras 

vocab_size = 8763
model3 = keras.Sequential(
    [
        keras.Input(shape=(1, 1, 512)),
        # layers.LSTM(4, input_shape=(30000,1,1,512),activation='relu', recurrent_activation='sigmoid',dropout=0.2),
     	  layers.Dense(256, activation='relu'),
        layers.Dense(64, activation='linear'),
        layers.Dense(64, activation='linear'),
        layers.Dense(vocab_size, activation='softmax')
    ],
    name='Generator2',
)


In [ ]:
model3.summary()

Model: "Generator2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_6 (Dense)              (None, 1, 1, 256)         131328    
_________________________________________________________________
dense_7 (Dense)              (None, 1, 1, 64)          16448     
_________________________________________________________________
dense_8 (Dense)              (None, 1, 1, 64)          4160      
_________________________________________________________________
dense_9 (Dense)              (None, 1, 1, 8763)        569595    
Total params: 721,531
Trainable params: 721,531
Non-trainable params: 0
_________________________________________________________________


In [ ]:
g1_optimizer = keras.optimizers.Adam(learning_rate=0.0004)

# Instantiate a loss function.
loss_fn1 = keras.losses.BinaryCrossentropy(from_logits=True)

def train_generator1(batch, epoch):
    
    
    batch_size = 64    
    with tf.GradientTape() as tape:
      # Decode them to fake images
      generated_caption = model3(batch) 
      generated_caption=tf.squeeze(generated_caption, axis=1)
      # generated_caption = layers.Reshape((batch_size,1,8763), input_dim = (batch_size, 1, 1, 8763))(generated_caption)
      #lstm = tf.keras.layers.LSTM(2)
      se2 = Dropout(0.5)(generated_caption)
      generated_caption = Dense(256, activation='linear')(se2)
      #generated_caption = LSTM(256)(se2)
      #generated_caption = np.expand_dims(generated_caption, axis=0)
      generated_caption = tf.expand_dims(generated_caption, axis=1)

      #generated_caption = tf.convert_to_tensor(generated_caption)
    
      # if (epoch == 468):
        # Z = tf.convert_to_tensor(GAN_Train_list2[epoch*batch_size:])
        # labels = tf.ones((48, 1), dtype=float)  
      # else:
      Z = tf.convert_to_tensor(GAN2_Train_list1[epoch*batch_size:(epoch+1)*(batch_size)])
      labels = tf.ones((batch_size, 1), dtype=float)
      generate = tf.concat([generated_caption, Z], axis = -1)
      #input - bs*1*1*512
    
      predictions = model2(generate)
      loss1 = loss_fn1(labels, predictions)
    grads = tape.gradient(loss1, model3.trainable_weights)
    g1_optimizer.apply_gradients(zip(grads, model3.trainable_weights))
    
    return loss1

In [ ]:
batch_size = 64
dataset = tf.data.Dataset.from_tensor_slices(GAN2_Train_list)
dataset = dataset.shuffle(buffer_size=1024).batch(batch_size)

In [ ]:

def fit1():
  
    # Losses & scores
    losses_g = []
    
    epochs = 25
    for epoch in range(epochs):
      i = 0
      for _, batch_X in enumerate(dataset):
        
      # for batch_X in ds.batch_iterator(batch_size=64):
        
        loss_g = train_generator1(batch_X, i)
        print(i)
        i = i + 1
        # Record losses & scores
        losses_g.append(loss_g)
        #losses_d.append(loss_d)
        #real_scores.append(real_score)
        #fake_scores.append(fake_score)
        
        # Log losses & scores (last batch)
      print("Loss After epoch")
      print(loss_g)  
      #print("Epoch [{}/{}], loss_g: {:.4f},  real_score: {:.4f}, fake_score: {:.4f}".format(
            #epoch+1, epochs, loss_g)) #loss_d, real_score, fake_score))
        # Save generated images
        #save_samples(epoch+start_idx, fixed_latent, show=False)
    
    return losses_g #, losses_d, real_scores, fake_scores

In [ ]:
loss_gen1 = fit1()

74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
Loss After epoch
tf.Tensor(0.6931472, shape=(), dtype=float32)
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
Loss After epoch
tf.Tensor(0.6931472, shape=(), dtype=float32)
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
Loss After epoch
tf.Tensor(0.6931472, shape=(), dtype=float32)
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
7

In [ ]:
model3.save_weights('/gdrive/Shared drives/BTP_Group2/checkpoints/my_checkpoint_model3')
model3.load_weights('/gdrive/Shared drives/BTP_Group2/checkpoints/my_checkpoint_model3')

In [ ]:
predictions = model3()
generated_caption = LSTM()(predictions)

In [ ]:
#######BLEU Score #######
# score=sentence_bleu(referernces, candidates)
# score2=sentence_bleu(referernces, candidates,weights=(0,0,0,1))


from nltk.translate.bleu_score import sentence_bleu
reference = [['this', 'is', 'a', 'test'], ['this', 'is' 'test']]
candidate = ['this', 'is', 'a', 'test']
score = sentence_bleu(reference, candidate)
print(score)

from nltk.translate.bleu_score import sentence_bleu
reference = [['this', 'is', 'a', 'test'], ['this', 'is' 'test']]
candidate = ['this', 'is', 'a', 'test']
score = sentence_bleu(reference, candidate)
print(score)
